# 01 - Preparación de Dataset para Detección de EPP

Este notebook guía el proceso de preparación del dataset para entrenar modelos de detección de EPP.

## Objetivos:
- Recolectar y organizar imágenes de entorno industrial
- Anotar EPPs usando herramientas como Roboflow, LabelImg o CVAT
- Preparar dataset en formato YOLO y Azure Custom Vision
- Realizar data augmentation para mejorar robustez

## EPPs a Detectar:
1. **Gafas de seguridad** (obligatorio)
2. **Zapatos de seguridad** (obligatorio)
3. **Traje/overol verde o cotona blanca/azul** (obligatorio)
4. **Mascarilla/respirador químico** (condicional)
5. **Guantes resistentes a químicos** (condicional)

In [ ]:
# Instalación de dependencias
!pip install roboflow pillow opencv-python albumentations pandas matplotlib

In [ ]:
import os
import yaml
from pathlib import Path
import shutil
from PIL import Image
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dotenv import load_dotenv

# Cargar variables de entorno
load_dotenv('../.env')

# Configuración
PROJECT_ROOT = Path('..').resolve()
DATA_DIR = PROJECT_ROOT / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
MODELS_DIR = DATA_DIR / 'models'

# Crear directorios
for dir_path in [RAW_DIR, PROCESSED_DIR, MODELS_DIR]:
    dir_path.mkdir(parents=True, exist_ok=True)

print(f"Directorios creados en: {DATA_DIR}")

## Paso 1: Fuentes de Datos

### Opción A: Dataset Público (Inicial)
Puedes empezar con datasets públicos de PPE/EPP:
- Roboflow Universe: https://universe.roboflow.com/search?q=ppe
- Dataset de Kaggle sobre PPE detection
- Open Images Dataset (con filtros de seguridad industrial)

### Opción B: Dataset Personalizado (Recomendado para producción)
Captura imágenes de TU planta de lixiviación:
1. Toma 500-1000 fotos del entorno real
2. Incluye variaciones: diferentes ángulos, iluminación, personas
3. Captura casos positivos (con EPP) y negativos (sin EPP)
4. Incluye los colores específicos: overol verde, cotona blanca/azul

In [ ]:
# Opción A: Descargar dataset de Roboflow
from roboflow import Roboflow

ROBOFLOW_API_KEY = os.getenv('ROBOFLOW_API_KEY')

if ROBOFLOW_API_KEY:
    rf = Roboflow(api_key=ROBOFLOW_API_KEY)
    
    # Ejemplo: descargar dataset público de PPE
    # Busca en https://universe.roboflow.com un dataset de PPE que te guste
    # project = rf.workspace("workspace-name").project("project-name")
    # dataset = project.version(1).download("yolov8", location=str(RAW_DIR))
    
    print("Configura tu proyecto de Roboflow para descargar dataset")
else:
    print("⚠️ ROBOFLOW_API_KEY no configurada. Añádela al archivo .env")
    print("Puedes obtener una API key gratuita en: https://roboflow.com")

## Paso 2: Estructura del Dataset

Organizaremos el dataset en formato estándar para YOLO y Azure:

```
data/
├── raw/                  # Datos originales sin procesar
├── processed/
│   ├── yolo/            # Formato YOLO (para open source)
│   │   ├── images/
│   │   │   ├── train/
│   │   │   ├── val/
│   │   │   └── test/
│   │   ├── labels/
│   │   │   ├── train/
│   │   │   ├── val/
│   │   │   └── test/
│   │   └── data.yaml
│   └── azure/           # Formato Azure Custom Vision
│       ├── images/
│       └── annotations.csv
└── models/              # Modelos entrenados
```

In [ ]:
# Crear estructura de directorios
YOLO_DIR = PROCESSED_DIR / 'yolo'
AZURE_DIR = PROCESSED_DIR / 'azure'

for split in ['train', 'val', 'test']:
    (YOLO_DIR / 'images' / split).mkdir(parents=True, exist_ok=True)
    (YOLO_DIR / 'labels' / split).mkdir(parents=True, exist_ok=True)

(AZURE_DIR / 'images').mkdir(parents=True, exist_ok=True)

print("✅ Estructura de directorios creada")

## Paso 3: Clases de EPP

Definimos las clases que detectará nuestro modelo:

In [ ]:
# Definición de clases
PPE_CLASSES = {
    0: 'safety_glasses',      # Gafas de seguridad
    1: 'safety_shoes',         # Zapatos de seguridad
    2: 'green_coverall',       # Overol verde
    3: 'white_coat',           # Cotona blanca
    4: 'blue_coat',            # Cotona azul
    5: 'chemical_mask',        # Mascarilla química
    6: 'chemical_gloves',      # Guantes químicos
    7: 'person_no_ppe',        # Persona sin EPP (para detección negativa)
}

# Crear archivo data.yaml para YOLO
data_yaml = {
    'path': str(YOLO_DIR.absolute()),
    'train': 'images/train',
    'val': 'images/val',
    'test': 'images/test',
    'nc': len(PPE_CLASSES),  # número de clases
    'names': list(PPE_CLASSES.values())
}

with open(YOLO_DIR / 'data.yaml', 'w') as f:
    yaml.dump(data_yaml, f, default_flow_style=False)

print("✅ Archivo data.yaml creado")
print(f"Clases: {list(PPE_CLASSES.values())}")

## Paso 4: Herramientas de Anotación

Para anotar tus imágenes personalizadas, usa una de estas herramientas:

### 1. Roboflow (Recomendado - Cloud)
- **Ventajas**: Interface intuitiva, auto-labeling con IA, augmentation automática
- **Link**: https://roboflow.com
- **Costo**: Plan gratuito hasta 10,000 imágenes
- **Export**: Directo a YOLO, Azure, etc.

### 2. LabelImg (Local - Open Source)
```bash
pip install labelImg
labelImg
```

### 3. CVAT (Local/Cloud - Open Source)
- **Link**: https://cvat.org
- **Ventajas**: Colaborativo, potente, gratis

### Consejos de Anotación:
1. **Precisión**: Dibuja bounding boxes ajustados al EPP
2. **Consistencia**: Usa los mismos nombres de clase siempre
3. **Casos difíciles**: Incluye oclusiones parciales, diferentes ángulos
4. **Balance**: Asegura que todas las clases tengan similar cantidad de ejemplos

## Paso 5: Data Augmentation

Aumentar el dataset con transformaciones para mejorar la robustez del modelo:

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Definir transformaciones de augmentation
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.3),
    A.RandomGamma(p=0.3),
    A.Blur(blur_limit=3, p=0.2),
    A.GaussNoise(p=0.2),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=15, p=0.5),
    A.HueSaturationValue(p=0.3),
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

print("✅ Pipeline de augmentation configurado")
print("Transformaciones: Flip, Brightness, Blur, Noise, Rotation, Color")

In [ ]:
# Ejemplo de aplicación de augmentation
def augment_image(image_path, bboxes, class_labels, num_augmentations=3):
    """
    Aplica data augmentation a una imagen y sus bounding boxes
    
    Args:
        image_path: Ruta a la imagen original
        bboxes: Lista de bounding boxes en formato YOLO [x_center, y_center, width, height]
        class_labels: Lista de etiquetas de clase para cada bbox
        num_augmentations: Número de versiones augmentadas a generar
    
    Returns:
        Lista de tuplas (imagen_augmentada, bboxes_augmentados, labels)
    """
    image = cv2.imread(str(image_path))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    augmented_samples = []
    
    for i in range(num_augmentations):
        transformed = transform(image=image, bboxes=bboxes, class_labels=class_labels)
        augmented_samples.append((
            transformed['image'],
            transformed['bboxes'],
            transformed['class_labels']
        ))
    
    return augmented_samples

# Ejemplo de uso (comentado hasta que tengas imágenes)
# augmented = augment_image('ruta/a/imagen.jpg', [[0.5, 0.5, 0.3, 0.4]], [0])
print("✅ Función de augmentation lista")

## Paso 6: Análisis del Dataset

Una vez que tengas imágenes anotadas, analiza la distribución:

In [ ]:
def analyze_dataset(labels_dir):
    """
    Analiza la distribución de clases en el dataset
    """
    class_counts = {class_name: 0 for class_name in PPE_CLASSES.values()}
    total_boxes = 0
    
    label_files = list(Path(labels_dir).glob('*.txt'))
    
    for label_file in label_files:
        with open(label_file, 'r') as f:
            for line in f:
                class_id = int(line.split()[0])
                class_name = PPE_CLASSES[class_id]
                class_counts[class_name] += 1
                total_boxes += 1
    
    # Crear DataFrame
    df = pd.DataFrame([
        {'Clase': k, 'Cantidad': v, 'Porcentaje': v/total_boxes*100 if total_boxes > 0 else 0}
        for k, v in class_counts.items()
    ])
    
    # Visualizar
    plt.figure(figsize=(12, 6))
    plt.bar(df['Clase'], df['Cantidad'])
    plt.xlabel('Clase de EPP')
    plt.ylabel('Número de instancias')
    plt.title('Distribución de Clases en el Dataset')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
    
    print("\n📊 Estadísticas del Dataset:")
    print(df.to_string(index=False))
    print(f"\nTotal de anotaciones: {total_boxes}")
    print(f"Total de imágenes: {len(label_files)}")
    
    return df

# Descomentar cuando tengas labels
# df_train = analyze_dataset(YOLO_DIR / 'labels' / 'train')
print("✅ Función de análisis lista")

## Paso 7: División Train/Val/Test

Recomendación estándar: 70% train, 20% validation, 10% test

In [ ]:
from sklearn.model_selection import train_test_split

def split_dataset(images_dir, labels_dir, train_ratio=0.7, val_ratio=0.2, test_ratio=0.1):
    """
    Divide el dataset en train/val/test
    """
    assert abs(train_ratio + val_ratio + test_ratio - 1.0) < 1e-5, "Las proporciones deben sumar 1"
    
    # Obtener lista de imágenes
    image_files = list(Path(images_dir).glob('*.jpg')) + list(Path(images_dir).glob('*.png'))
    
    # Primera división: train y temp (val+test)
    train_files, temp_files = train_test_split(
        image_files, 
        train_size=train_ratio, 
        random_state=42
    )
    
    # Segunda división: val y test
    val_ratio_adjusted = val_ratio / (val_ratio + test_ratio)
    val_files, test_files = train_test_split(
        temp_files,
        train_size=val_ratio_adjusted,
        random_state=42
    )
    
    print(f"📊 División del dataset:")
    print(f"  Train: {len(train_files)} imágenes ({train_ratio*100:.0f}%)")
    print(f"  Val:   {len(val_files)} imágenes ({val_ratio*100:.0f}%)")
    print(f"  Test:  {len(test_files)} imágenes ({test_ratio*100:.0f}%)")
    
    return train_files, val_files, test_files

# Ejemplo de uso (descomentar cuando tengas imágenes)
# train, val, test = split_dataset(RAW_DIR / 'images', RAW_DIR / 'labels')
print("✅ Función de división lista")

## Resumen y Próximos Pasos

### ✅ Has completado:
1. Configuración de estructura de datos
2. Definición de clases de EPP
3. Pipeline de data augmentation
4. Herramientas de análisis

### 📝 Tareas pendientes:
1. **Recolectar/descargar imágenes** de EPP en entorno industrial
2. **Anotar imágenes** usando Roboflow, LabelImg o CVAT
3. **Aplicar augmentation** para expandir el dataset
4. **Analizar distribución** de clases
5. **Dividir en train/val/test**

### ➡️ Siguiente notebook:
- **02_model_training_azure.ipynb**: Entrenar modelo con Azure Custom Vision
- **03_model_training_opensource.ipynb**: Entrenar modelo con YOLOv8

### 💡 Recursos útiles:
- [Roboflow Universe - PPE Datasets](https://universe.roboflow.com/search?q=ppe)
- [YOLOv8 Documentation](https://docs.ultralytics.com/)
- [Azure Custom Vision](https://www.customvision.ai/)
- [LabelImg Tutorial](https://github.com/heartexlabs/labelImg)

In [ ]:
# Guardar configuración para próximos notebooks
config = {
    'project_root': str(PROJECT_ROOT),
    'data_dir': str(DATA_DIR),
    'yolo_dir': str(YOLO_DIR),
    'azure_dir': str(AZURE_DIR),
    'classes': PPE_CLASSES,
    'num_classes': len(PPE_CLASSES)
}

with open(DATA_DIR / 'config.yaml', 'w') as f:
    yaml.dump(config, f)

print("✅ Configuración guardada en data/config.yaml")